In [20]:
import tensorflow as tf
from datetime import datetime

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [23]:
class Dense(tf.Module):
    def __init__(self, in_features = 3, out_features = 1, name=None):
        super().__init__(name=name)
        self.w = tf.Variable(([[-3.14,-2.31,2.16]]), name = 'w')
    def __call__(self, x):
        y = tf.matmul(self.w, x)
        return tf.nn.sigmoid(y)

In [24]:
my_module = Dense(in_features=3,out_features=1,name = 'mymodule')
my_module(tf.constant([[0.0288],[-0.3256],[0.5925]]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.8745173]], dtype=float32)>

In [25]:
class Dense(tf.Module):
    def __init__(self, in_features, out_features, name=None):
        super().__init__(name=name)
        self.w = tf.Variable(tf.random.normal([in_features,out_features]), name = 'w')
        self.b = tf.Variable(tf.zeros([out_features]), name = 'b')
    def __call__(self, x):
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.sigmoid(y)
    
    
class SequentialModule(tf.Module):
    def __init__(self, name = None):
        super().__init__(name=name)
        
        self.dense_1 = Dense(in_features = 3, out_features = 3)
        self.dense_2 = Dense(in_features = 3, out_features = 2)
     
    @tf.function
    def __call__(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)
    
my_model = SequentialModule(name = 'mymodel')
my_model(tf.constant([[2.0,2.0,2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.87050956, 0.8717431 ]], dtype=float32)>

In [26]:
chkp_path = 'my_checkpoint'
checkpoint = tf.train.Checkpoint(model = my_model)
checkpoint.write(chkp_path)

'my_checkpoint'

In [27]:
tf.train.list_variables(chkp_path)

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('model/dense_1/b/.ATTRIBUTES/VARIABLE_VALUE', [3]),
 ('model/dense_1/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 3]),
 ('model/dense_2/b/.ATTRIBUTES/VARIABLE_VALUE', [2]),
 ('model/dense_2/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 2])]

In [29]:
stamp = datetime.now().strftime('%Y%m%d-%H%M%S')
logdir = 'log/func/%s' % stamp
writer = tf.summary.create_file_writer(logdir)

new_model = SequentialModule()
tf.summary.trace_on(graph = True, profiler = True)
z = print(new_model(tf.constant([[2.0,2.0,2.0]])))
with writer.as_default():
    tf.summary.trace_export(
    name = 'my_func_trace',
    step = 0,
    profiler_outdir = logdir)

tf.Tensor([[0.4003772 0.2336638]], shape=(1, 2), dtype=float32)


In [17]:
%tensorboard --logdir logs/func  

Reusing TensorBoard on port 6006 (pid 13408), started 0:01:27 ago. (Use '!kill 13408' to kill it.)

In [30]:
tf.saved_model.save(my_model,'bdmi_model')

INFO:tensorflow:Assets written to: bdmi_model\assets


In [31]:
new_model = tf.saved_model.load('bdmi_model')

In [32]:
print(my_model([[2.0,2.0,2.0]]))

tf.Tensor([[0.87050956 0.8717431 ]], shape=(1, 2), dtype=float32)


In [ ]:
def a(x,y,b):
    return result = tf.matmul(x,y)+b
gragh = tf.function(a)

x1 = tf.constant([[1.0,2.0]])
y1 = tf.constant([[2.0],[3.0]])
b1 = 